# Image Reduction for Imaging Project
This notebook illustrates how to use `ccdred_sb`. To run this notebook you will need to download the raw artificial images from the [companion website for my textbook *A Practical Guide to Observational Astronomy*](https://mshaneburns.github.io/ObsAstro/). Place the images in a subdirectory called `images`. 

In this example, I assume that we don't need to make a dark current correction and that all of the object images are all taken in the same bandpass so we don't need separate flats for each bandpass.

## Make lists of files to process
The functions in `ccdred_sb` require a text file containing a list of the images to be proccesed. You can create the text files anyway you like, but I used the following unix commands to create three lists. The file `zero_list.txt` contains all of the bias frame file names, `flat_list.txt` contains the flat field file names, and `object_list.txt` contains the names of the files to be processed. The unix session starts in the directory contaning this notebook (called `myProject` in this case), and `ccdred_sb`.

I deleted all of the processed images in the `images` directory before executing these commands.

```unix
myProject % cd images
images % ls -1 bias*.fits > ../zero_list.txt
images % ls -1 flat*.fits > ../flat_list.txt
images % ls -1 star*.fits > ../object_list.txt
images % ls -1 gal*.fits >> ../object_list.txt
images % 
```

## Process the images
In order to process the data I will

1. Create a master bias frame called `Zero.fits`
2. Create master flat frame called `Flat.fit`. If we needed to process images in other bandpasses we would need a flat frame for each bandpass.
3. Run `ccdproc` to correct the object images. 

I also have **DS9** open on my desktop to examine the images after each step to make sure the process worked correctly. I find this easier than displaying the image using `imshow()`.

I start by importing the needed modules

In [1]:
import ccdred_sb as cc

### Create master bias frame
I use `cc.zerocombine()`. You can use `help()` to understand its arguments and how it works.

In [2]:
help(cc.zerocombine)

Help on function zerocombine in module ccdred_sb:

zerocombine(imdir='images/', input_list='zero_list.txt', output='Zero')
    Make master bias image.
    
    Like IRAF's `zerocombine`. It currently combines the frames by
    averaging all of the bias frames listed in the `input_list`. It writes
    out a FITS file with the name given by the `output` parameter. Prints the
    `mean` and `sigma` (standard deviation) of the output file's pixel values.
    
    Parameters
    ----------
    imdir : str
        Path to image directory (must include trailing '/')
    input_list : str
        Name of file containing list of filenames
    output : str
        Name for the output FITS file.
    
    Returns
    -------
    fout_name : str
        Name with `.fits` extension of created file



We won't have to change any of the default arguments since all of our images are in the `images` directory and the list of bias frames is called `zero_list.txt`.

In [3]:
cc.zerocombine()

Reading bias01.fits
Reading bias02.fits
Reading bias03.fits
Reading bias04.fits
Reading bias05.fits
Reading bias06.fits
Reading bias07.fits
Reading bias08.fits
Reading bias09.fits
Reading bias10.fits
zerocombine output file name: Zero.fits
    mean pixel value = 349.995697
    standard deviation = 2.611144


'Zero.fits'

I examined the image using **DS9** and it looks like what I expected and the signal level is about 942. This is also what I expected. 

## Create the master flat field 
These images are called flat field images, but they are efficiency maps of the CCD and optical system. If we were processing data in more than one bandpass we would need to create one for each filter. Let's use `help()` to determine how we need to reset parameters for the `cc.flatcombine()` function.

In [4]:
help(cc.flatcombine)

Help on function flatcombine in module ccdred_sb:

flatcombine(imdir='images/', input_list='flat_list.txt', output='Flat', zero_sub=True, zero_file='Zero.fits', dark_correction=False, dark_file='Dark.fits')
    Like IRAF's `flatcombine`.
    
    It currently works by: 
    
    1. Subtract `zero_file` from each file in `input_list` if `zero_sub = True`
    2. Compute median pixel value for each file in `input_list`
    3. Create the master efficiency map file with name specified by `output`
    
    Parameters
    ----------
    imdir : str
        Path to image directory (must include trailing '/')
    input_list : str
        Name of file containing list of filenames
    output : str
        Name for the output FITS file.
    zero_sub : boolean
        Flag for bias subtraction
    zero_file : str
        Name of bias file for bias subraction        
    dark_correction : boolean
        Flag for dark subtraction
    dark_file : str
        Name of dark file for dark subraction     

Again we don't need to change any of the fuctions parameter.

In [5]:
cc.flatcombine()

Reading flat01.fits
Reading flat02.fits
Reading flat03.fits
Reading flat04.fits
Reading flat05.fits
Flat image meadian = 56840.244102
Flat image meadian = 54879.661381
Flat image meadian = 52918.781507
Flat image meadian = 50959.597832
Flat image meadian = 48999.796750
         File name set to: Flat.fits
         'FILTER' set to 'none'
flatcombine output file name: Flat.fits


'Flat.fits'

The file looks like what I expect an efficiency map should. It's average pixel value is about 1.0 which it should be for an efficiency map.

By examining with **DS9** it seems that both of these images are correct as well.

# Proccess the object images
Again I start by getting help for the function.

In [6]:
help(cc.ccdproc)

Help on function ccdproc in module ccdred_sb:

ccdproc(imdir='images/', input_list='object_list.txt', output_list=None, zero_file='Zero.fits', flat_file='Flat.fits', dark_correction=False, dark_file='Dark.fits')
    Like IRAF's `ccdproc`.
    
    It currently works by: 
    
    1. Subtract `zero_file` from each file in the `input_list` 
    2. Divide by `flat_file` from each file in the `input_list`
    3. Write processed FITS files with name specified by `output_list`
    
    Parameters
    ----------
    input_list : str
        Name of file containing list of input file names
    output_list : str
        Name of file containing output file names. If `output_list = None` the
        filenames are the same as those in `input_list` with `_p` appended.
    zero_file : str
        Name of bias correction file
    flat_file : str
        Name of flat correction file       
    dark_correction : boolean
        applies dark correction if True
    dark_file : str
        dark current im

We don't need to change any function parameters.

In [7]:
cc.ccdproc()

Running ccdproc
Reading stars01.fits
Reading stars02.fits
Reading stars03.fits
Reading stars04.fits
Reading stars05.fits
Reading gal_new.fits
Reading gal_ref.fits
Using zero image: Zero.fits
Using flat image: Flat.fits
Writing file stars01_p.fits

Using zero image: Zero.fits
Using flat image: Flat.fits
Writing file stars02_p.fits

Using zero image: Zero.fits
Using flat image: Flat.fits
Writing file stars03_p.fits

Using zero image: Zero.fits
Using flat image: Flat.fits
Writing file stars04_p.fits

Using zero image: Zero.fits
Using flat image: Flat.fits
Writing file stars05_p.fits

Using zero image: Zero.fits
Using flat image: Flat.fits
Writing file gal_new_p.fits

Using zero image: Zero.fits
Using flat image: Flat.fits
Writing file gal_ref_p.fits



['stars01_p.fits',
 'stars02_p.fits',
 'stars03_p.fits',
 'stars04_p.fits',
 'stars05_p.fits',
 'gal_new_p.fits',
 'gal_ref_p.fits']

The final step is to examine the images to make sure they look reasonable. The final processed object images have the same name as the raw images, but with an `_p` appended. 